## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Torbay,CA,47.6666,-52.7314,41.76,79,20,6.91,few clouds
1,1,Vaini,TO,-21.2000,-175.2000,77.16,88,40,9.22,scattered clouds
2,2,Mataura,NZ,-46.1927,168.8643,49.66,96,100,12.59,light rain
3,3,Aras,ES,42.5617,-2.3560,56.98,100,100,1.14,mist
4,4,Ushuaia,AR,-54.8000,-68.3000,51.46,46,0,19.57,clear sky


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? (F) "))
max_temp = float(input("What is the maximum temperature you would like for your trip? (F) "))

What is the minimum temperature you would like for your trip? (F) 75
What is the maximum temperature you would like for your trip? (F) 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Vaini,TO,-21.2000,-175.2000,77.16,88,40,9.22,scattered clouds
12,12,Sao Felix Do Xingu,BR,-6.6447,-51.9950,83.07,55,93,0.89,overcast clouds
14,14,Rikitea,PF,-23.1203,-134.9692,75.02,85,80,21.12,light rain
16,16,San Andres,CO,12.5847,-81.7006,84.22,79,75,13.80,light rain
31,31,Galle,LK,6.0367,80.2170,77.14,89,86,2.46,overcast clouds
36,36,Conde,BR,-7.2597,-34.9075,78.26,73,0,13.80,clear sky
39,39,Wewak,PG,-3.5534,143.6268,81.30,83,51,4.29,broken clouds
40,40,Spanish Town,JM,17.9911,-76.9574,85.62,72,20,12.66,few clouds
43,43,Puerto Ayora,EC,-0.7393,-90.3518,78.76,82,96,9.57,overcast clouds
44,44,Kavaratti,IN,10.5669,72.6420,81.68,72,81,2.13,broken clouds


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Vaini,TO,-21.2000,-175.2000,77.16,88,40,9.22,scattered clouds
12,12,Sao Felix Do Xingu,BR,-6.6447,-51.9950,83.07,55,93,0.89,overcast clouds
14,14,Rikitea,PF,-23.1203,-134.9692,75.02,85,80,21.12,light rain
16,16,San Andres,CO,12.5847,-81.7006,84.22,79,75,13.80,light rain
31,31,Galle,LK,6.0367,80.2170,77.14,89,86,2.46,overcast clouds


In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vaini,TO,77.16,scattered clouds,-21.2000,-175.2000,
12,Sao Felix Do Xingu,BR,83.07,overcast clouds,-6.6447,-51.9950,
14,Rikitea,PF,75.02,light rain,-23.1203,-134.9692,
16,San Andres,CO,84.22,light rain,12.5847,-81.7006,
31,Galle,LK,77.14,overcast clouds,6.0367,80.2170,
36,Conde,BR,78.26,clear sky,-7.2597,-34.9075,
39,Wewak,PG,81.30,broken clouds,-3.5534,143.6268,
40,Spanish Town,JM,85.62,few clouds,17.9911,-76.9574,
43,Puerto Ayora,EC,78.76,overcast clouds,-0.7393,-90.3518,
44,Kavaratti,IN,81.68,broken clouds,10.5669,72.6420,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat}, {lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [44]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
for index, row in hotel_df.iterrows():
    hotel_df.replace("", nan_value, inplace=True)

clean_hotel_df = hotel_df.dropna()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vaini,TO,77.16,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
12,Sao Felix Do Xingu,BR,83.07,overcast clouds,-6.6447,-51.9950,Rio Xingu Camping
14,Rikitea,PF,75.02,light rain,-23.1203,-134.9692,People ThankYou
16,San Andres,CO,84.22,light rain,12.5847,-81.7006,Hotel Arena Blanca
31,Galle,LK,77.14,overcast clouds,6.0367,80.2170,Amangalla
36,Conde,BR,78.26,clear sky,-7.2597,-34.9075,Pousada Beija Flor
39,Wewak,PG,81.30,broken clouds,-3.5534,143.6268,In Wewak Boutique Hotel
40,Spanish Town,JM,85.62,few clouds,17.9911,-76.9574,Shan's Cloud
43,Puerto Ayora,EC,78.76,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
44,Kavaratti,IN,81.68,broken clouds,10.5669,72.6420,Hotels in Lakshadweep Islands


In [46]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [52]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [54]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))